<a href="https://colab.research.google.com/github/gandhimonil9823/Covid_Stock_Analysis/blob/main/Covid_Stock.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!apt-get install -q libgeos-3.5.0
!apt-get install -q libgeos-dev
!pip install -q https://github.com/matplotlib/basemap/archive/master.zip

Reading package lists...
Building dependency tree...
Reading state information...
E: Unable to locate package libgeos-3.5.0
E: Couldn't find any package by glob 'libgeos-3.5.0'
E: Couldn't find any package by regex 'libgeos-3.5.0'
Reading package lists...
Building dependency tree...
Reading state information...
Suggested packages:
  libgdal-doc
The following NEW packages will be installed:
  libgeos-dev
0 upgraded, 1 newly installed, 0 to remove and 12 not upgraded.
Need to get 73.1 kB of archives.
After this operation, 486 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libgeos-dev amd64 3.6.2-1build2 [73.1 kB]
Fetched 73.1 kB in 1s (127 kB/s)
Selecting previously unselected package libgeos-dev.
(Reading database ... 144786 files and directories currently installed.)
Preparing to unpack .../libgeos-dev_3.6.2-1build2_amd64.deb ...
Unpacking libgeos-dev (3.6.2-1build2) ...
Setting up libgeos-dev (3.6.2-1build2) ...
Processing trigge

In [3]:
!pip install -q pyproj==1.9.6

     |████████████████████████████████| 2.8MB 3.3MB/s 


In [4]:
!pip install pathpy

     |████████████████████████████████| 235kB 3.5MB/s 


In [187]:
import pandas as pd
import matplotlib.pyplot as plt
import os
import statistics 
import numpy as np
from mpl_toolkits.basemap import Basemap
%matplotlib inline
import pathpy as pp
from geopy import distance
from geopy.distance import geodesic
import itertools
import math

In [6]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [7]:
# Merging the Stock Indexes and the currency data with rows as dates and columns as index values
# Preprocessing the data sets with online steps for data processing 
# Figuring a way to convert the unemployment data into county wise data - probably using a probability distribution
# Deal with missing stock data for weekends and holidays

Stock Data: Pre-processing


In [8]:
# Reading CSVs for three stock indices
df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/DJI.csv")
df_DJI = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/NASDAQ.csv")
df_NASDAQ = df[["Date", "Adj Close", "Volume"]]

df = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/S&P500.csv")
df_SP500 = df[["Date", "Adj Close", "Volume"]]

df_covid_clean_complete = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/covid_19_clean_complete.csv")

df_centroid = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/pre_processed_centroid.csv")

df_with_missing = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/stock_closing_prices_188.csv")
df_with_missing_historical = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/stock_data/historical_data_filled_188.csv")

In [9]:
# Rename columns in each index in order to merge the indices 
df_NASDAQ = df_NASDAQ.rename(columns={'Adj Close': 'Adj_Close_NASDAQ', 'Volume': 'Volume_NASDAQ'})
df_SP500 = df_SP500.rename(columns={'Adj Close': 'Adj_Close_SP500', 'Volume': 'Volume_SP500'})
df_DJI = df_DJI.rename(columns={'Adj Close': 'Adj_Close_DJI', 'Volume': 'Volume_DJI'})

In [10]:
# Merging data of three stock indices based on column: 'Date' (merging two DFs at once)
df_Merged = pd.merge(df_SP500, df_DJI, on=['Date'])
df_stock_data = pd.merge(df_Merged, df_NASDAQ, on=['Date'])

In [11]:
df_stock_data.head()

,Date,Adj_Close_SP500,Volume_SP500,Adj_Close_DJI,Volume_DJI,Adj_Close_NASDAQ,Volume_NASDAQ
0,2015-10-26,2071.179932,3385800000,17623.050781,116660000,5034.700195,1758690000
1,2015-10-27,2065.889893,4216880000,17581.429688,113960000,5030.149902,1986840000
2,2015-10-28,2090.350098,4698110000,17779.519531,115630000,5095.689941,2141130000
3,2015-10-29,2089.409912,4008940000,17755.800781,90300000,5074.270020,1928310000
4,2015-10-30,2079.360107,4256200000,17663.539063,149250000,5053.750000,2016390000


In [12]:
df_stock_data.shape

(1259, 7)

In [13]:
df_with_missing.head()

,Date,Adj_Close_SP500,Adj_Close_DJI,Adj_Close_NASDAQ
0,2020-01-22,3321.750000,29186.26953,9383.769531
1,2020-01-23,3325.540039,29160.08984,9402.480469
2,2020-01-24,3295.469971,28989.73047,9314.910156
3,2020-01-25,3278.098532,28837.62388,9256.004910
4,2020-01-26,3260.818664,28686.31538,9197.472167


In [14]:
df_with_missing.shape

(188, 4)

In [15]:
df_with_missing_historical

,Date,Adj_Close_SP500,Adj_Close_DJI,Adj_Close_NASDAQ
0,2019-07-18,2995.110107,27222.97070,8207.240234
1,2019-07-19,2976.610107,27154.19922,8146.490234
2,2019-07-20,2979.414105,27160.09833,8165.661554
3,2019-07-21,2982.220745,27165.99872,8184.877990
4,2019-07-22,2985.030029,27171.90039,8204.139648
...,...,...,...,...
183,2020-01-17,3329.620117,29348.09961,9388.940430
184,2020-01-18,3326.674152,29297.32497,9382.892916
185,2020-01-19,3323.730793,29246.63817,9376.849296
186,2020-01-20,3325.202147,29271.97060,9379.870619


# COVID-19 Data Loading and Preprocessing

In [16]:
# Function to load the covid_clean_complete dataset from the drive
def load_dataset_us_cases():  
  df_covid_clean_complete = pd.read_csv("/content/drive/Shared drives/Stock_Covid_Data/COVID_19/covid_19_clean_complete.csv")
  return df_covid_clean_complete

# Function to preprocess the clean_complete_dataset
def preprocess_us_cases_df():
  df_US_daily_cases = load_dataset_us_cases()

  # get the daily data only for US
  df_US_daily_cases = df_covid_clean_complete.loc[df_covid_clean_complete['Country/Region'] == "US"]
  
  df_US_daily_cases = df_US_daily_cases.reset_index(drop=True)

  # Added a column for new_cases daily by getting the differerence of the confirmed cases
  df_US_daily_cases["New_cases"] = df_US_daily_cases["Confirmed"].diff()
  
  df_US_daily_cases['New_cases'] = df_US_daily_cases['New_cases'].fillna(0)

  df_US_daily_cases = df_US_daily_cases.rename(columns={"Confirmed": "Total_Confirmed", "Deaths" : "Total_Deaths"})

  return df_US_daily_cases


# Standardize the COVID variables like New_cases, Total_cases, Total_deaths

In [ ]:
# Function to standardize the covid_variables like New_cases, Total_cases, Total_deaths
# Adds new columns to the df with the standarized values
# params - the dataframe, and the column name (COVID Variable) which has to be standarized
def covid_variable_impact_cal(dataframe_to_standardize, column_name):
  # col_mean = dataframe_to_standardize[column_name].mean()
  # col_std  = dataframe_to_standardize[column_name].std()

  col_name = "Standardized_Value_" + column_name
  dataframe_to_standardize[col_name] = dataframe_to_standardize[column_name]

  i = 0
  j = 0
  while i < len(dataframe_to_standardize):
    j = i
    if i > 7:
      col_mean = dataframe_to_standardize[column_name][j - 8: j - 1].mean()
      col_std  = statistics.stdev(dataframe_to_standardize[column_name][j - 8: j - 1])

      curr_daily_value = dataframe_to_standardize[column_name][i]

      formula_cal_value = (curr_daily_value - col_mean) / col_std

      dataframe_to_standardize.loc[i, col_name] = formula_cal_value

    i += 1
  dataframe_to_standardize[col_name].replace(np.inf, 0, inplace=True)
  return dataframe_to_standardize

In [ ]:
df_US_daily_cases = preprocess_us_cases_df()
df_US_daily_cases.head()



,Province/State,Country/Region,Lat,Long,Date,Total_Confirmed,Total_Deaths,Recovered,Active,WHO Region,New_cases
0,NaN,US,40.0,-100.0,2020-01-22,1,0,0,1,Americas,0.0
1,NaN,US,40.0,-100.0,2020-01-23,1,0,0,1,Americas,0.0
2,NaN,US,40.0,-100.0,2020-01-24,2,0,0,2,Americas,1.0
3,NaN,US,40.0,-100.0,2020-01-25,2,0,0,2,Americas,0.0
4,NaN,US,40.0,-100.0,2020-01-26,5,0,0,5,Americas,3.0


In [ ]:
df_standarized_total_confirmed = covid_variable_impact_cal(df_US_daily_cases, "Total_Confirmed")

df_standarized_total_confirmed.reset_index().plot(x='index', y='Standardized_Value_Total_Confirmed')    

df_standarized_Total_Deaths = covid_variable_impact_cal(df_US_daily_cases, "Total_Deaths")

df_standarized_Total_Deaths.reset_index().plot(x='index', y='Standardized_Value_Total_Deaths')

df_standarized_New_cases = covid_variable_impact_cal(df_US_daily_cases, "New_cases")

df_standarized_New_cases.reset_index().plot(x='index', y='Standardized_Value_New_cases')

# Complex Network Analysis

In [ ]:
# Using Centroid of all the long and lat
# def combinations_distance_df(df_original):
#     counties  = df_original["Combined_Key"]
    
#     list_of_counties_combinations = list(itertools.combinations(counties,2))

#     df_to_return = pd.DataFrame({"Combinations" : list_of_counties_combinations})

#     array_distances = []
#     for item in list_of_counties_combinations:
#         row_element1 = df_original.loc[df_original['Combined_Key'] == item[0]]
#         row_element2 = df_original.loc[df_original['Combined_Key'] == item[1]]
        
#         value1 = float(row_element1["Centroid_Distance"].tolist()[0])
#         value2 = float(row_element2["Centroid_Distance"].tolist()[0])

#         array_distances.append(abs(value1 - value2))

#     df_to_return["Distance"] = array_distances
#     return df_to_return    


# Using Two counties difference directly
def combinations_distance_df(df_original):
    counties  = df_original["Combined_Key"]

    list_of_counties_combinations = list(itertools.combinations(counties,2))

    df_to_return = pd.DataFrame({"Combinations" : list_of_counties_combinations})

    array_distances = []
    for item in list_of_counties_combinations:
        row_element1 = df_original.loc[df_original['Combined_Key'] == item[0]]
        row_element2 = df_original.loc[df_original['Combined_Key'] == item[1]]

        long1 = float(row_element1["Long_"].tolist()[0])
        lat1 = float(row_element1["Lat"].tolist()[0])
        
        long2 = float(row_element2["Long_"].tolist()[0])
        lat2 = float(row_element2["Lat"].tolist()[0])

        point_n = geodesic((lat1, long1),(lat2, long2)).miles
        
        array_distances.append(point_n)

    df_to_return["Distance"] = array_distances
    return df_to_return    

    

In [ ]:
df_centroid = df_centroid[df_centroid.New_cases != 0]

df_centroid = df_centroid.reset_index(drop=True)

groupby_daily = df_centroid.groupby(df_centroid["Date"])

i = 0
g = None
for name, group in groupby_daily:
# 2174
    if len(group) == 2174:
        fig = plt.figure(figsize=(16, 10))
        m_ = Basemap(llcrnrlon=-119,llcrnrlat=22,urcrnrlon=-64,urcrnrlat=49,
        projection='lcc',lat_1=33,lat_2=45,lon_0=-95)
        m_.readshapefile('/content/drive/Shared drives/Stock_Covid_Data/COVID_19/st99_d00', name='states')
        df_comb_counties_distance = combinations_distance_df(group)
        G_t = pp.Network(directed=False)
        for index_group, row_group in group.iterrows():
            if row_group["New_cases"] >= 0:
                G_t.add_node(str(row_group["Combined_Key"]))
        m_.scatter(group["Long_"].tolist(), group["Lat"].tolist(), latlon=True, s=25, c="red", marker='o')    
        for index, row in df_comb_counties_distance.iterrows():
            if row["Distance"] < 100:
                tuple_county = row["Combinations"]
                if str(tuple_county[0]) in G_t.nodes and str(tuple_county[1]) in G_t.nodes:
                    G_t.add_edge(str(tuple_county[0]), str(tuple_county[1]))
        plt.title(str(name), fontsize=20)
        
        g = G_t
#         print(name)


#         pp.visualisation.plot(G_t)    
    # print(len(group))
        
style = {'width': 1000, 
          'height': 1000
        }
pp.visualisation.export_html(g, 'my_temporal_network.html')
pp.visualisation.plot(g,**style)

## Computing Stock paramaters - Abnormal price and Volatality:

In [487]:
def compute_stock_params(covid_stock_values, precovid_stock_values):
  
  # Average of daily stock prices in the pre-COVID period (P_t-i)
  precovid_stocks_sum = precovid_stock_values[1::].sum()
  avg_precovid_stocks = precovid_stocks_sum[1::].div(188)

  # Differece between the daily stock prices in the COVID period and the average of daily stock prices in the pre-covid period (P_t - P_t-i)
  snp_diff = [daily_stocks - avg_precovid_stocks[0] for daily_stocks in precovid_stock_values['Adj_Close_SP500']]
  dji_diff = [daily_stocks - avg_precovid_stocks[1] for daily_stocks in precovid_stock_values['Adj_Close_DJI']]
  nasdaq_diff = [daily_stocks - avg_precovid_stocks[2] for daily_stocks in precovid_stock_values['Adj_Close_NASDAQ']]

  diff = pd.DataFrame(
    {'Adj_Close_SP500': snp_diff,
     'Adj_Close_DJI': dji_diff,
     'Adj_Close_NASDAQ': nasdaq_diff
    })
  
  # Standard deviatoon of the pre-COVID period
  sd_stock_indices = (diff**2 / 188)**0.5

  # Abnormal price  
  abnormal_price = (covid_stock_values -  avg_precovid_stocks) / sd_stock_indices;
  abnormal_price['Date'] = covid_stock_values['Date']

  print("covid_stock_values = ",covid_stock_values)
  # log(P_t/P_t-1)
  rt = pd.DataFrame([[0, 0, 0]], columns = ['Adj_Close_SP500', 'Adj_Close_DJI', 'Adj_Close_NASDAQ'])
  for i in range(1, len(covid_stock_values)):
    previous = covid_stock_values.loc[i-1,covid_stock_values.columns != 'Date']
    today = covid_stock_values.loc[i,covid_stock_values.columns != 'Date']
    temp = today.div(previous)
    rt = rt.append(temp, ignore_index=True)

  rt = rt.apply(np.log10)
  
  # Volatality of closing stock prices in the COVID period
  vol = rt**2

  rt['Date'] = covid_stock_values['Date']
  vol['Date'] = covid_stock_values['Date']
  
  return abnormal_price, rt, vol

In [488]:
abnormal_price, rt, vol = compute_stock_params(df_with_missing, df_with_missing_historical)
abnormal_price

covid_stock_values =             Date  Adj_Close_SP500  Adj_Close_DJI  Adj_Close_NASDAQ
0    2020-01-22      3321.750000    29186.26953       9383.769531
1    2020-01-23      3325.540039    29160.08984       9402.480469
2    2020-01-24      3295.469971    28989.73047       9314.910156
3    2020-01-25      3278.098532    28837.62388       9256.004910
4    2020-01-26      3260.818664    28686.31538       9197.472167
..          ...              ...            ...               ...
183  2020-07-23      3235.659912    26652.33008      10461.419920
184  2020-07-24      3215.629883    26469.89063      10363.179690
185  2020-07-25      3223.537100    26508.12833      10420.558030
186  2020-07-26      3231.463760    26546.42127      10478.254050
187  2020-07-27      3239.409912    26584.76953      10536.269530

[188 rows x 4 columns]


,Adj_Close_DJI,Adj_Close_NASDAQ,Adj_Close_SP500,Date
0,905.153,113.33,90.0023,2020-01-22
1,699.294,78.0225,63.8512,2020-01-23
2,754.057,79.7732,59.8895,2020-01-24
3,842.337,84.6297,58.6394,2020-01-25
4,981.11,90.9915,57.284,2020-01-26
...,...,...,...,...
183,-3.43808,27.6535,9.28907,2020-07-23
184,-4.70963,26.5285,8.43162,2020-07-24
185,-4.57059,27.4368,8.8984,2020-07-25
186,-4.26238,28.1141,9.23158,2020-07-26


In [481]:
rt

,Adj_Close_SP500,Adj_Close_DJI,Adj_Close_NASDAQ,Date
0,-inf,-inf,-inf,2020-01-22
1,0.000495,-0.000390,0.000865,2020-01-23
2,-0.003945,-0.002545,-0.004064,2020-01-24
3,-0.002295,-0.002285,-0.002755,2020-01-25
4,-0.002295,-0.002285,-0.002755,2020-01-26
...,...,...,...,...
183,-0.005384,-0.005723,-0.010042,2020-07-23
184,-0.002697,-0.002983,-0.004098,2020-07-24
185,0.001067,0.000627,0.002398,2020-07-25
186,0.001067,0.000627,0.002398,2020-07-26


In [482]:
vol

,Adj_Close_SP500,Adj_Close_DJI,Adj_Close_NASDAQ,Date
0,inf,inf,inf,2020-01-22
1,2.452600e-07,1.518903e-07,7.484104e-07,2020-01-23
2,1.556162e-05,6.475397e-06,1.651422e-05,2020-01-24
3,5.268656e-06,5.219874e-06,7.590543e-06,2020-01-25
4,5.268655e-06,5.219874e-06,7.590543e-06,2020-01-26
...,...,...,...,...
183,2.898400e-05,3.274706e-05,1.008392e-04,2020-07-23
184,7.272797e-06,8.898506e-06,1.679038e-05,2020-07-24
185,1.137672e-06,3.930256e-07,5.750154e-06,2020-07-25
186,1.137672e-06,3.930257e-07,5.750152e-06,2020-07-26
